<a href="https://colab.research.google.com/github/theabhinav0231/dive_into_deep_learning_book/blob/main/10_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>